In [1]:
import tensorflow as tf
import os
import splitfolders as sf
from pathlib import Path
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from sklearn.metrics import roc_curve, roc_auc_score
import math
import matplotlib.pyplot as plt
import numpy as np
import cv2
import PIL
from tensorflow.keras import layers, models
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, concatenate, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from keras.models import load_model
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
train_dir = r'A:\AI DB\LSPD\Videos\photos_from_videos_pret\train'
validation_dir = r'A:\AI DB\LSPD\Videos\photos_from_videos_pret\val'
test_dir = r'A:\AI DB\LSPD\Videos\photos_from_videos_pret\test'

# Define image dimensions and batch size
img_height, img_width = 224, 224
batch_size = 256

In [3]:
# Define data generators for training, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.5, 1.5] 
)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,                                          
    seed=10,
    class_mode='binary'
)

Found 430877 images belonging to 2 classes.
Found 23937 images belonging to 2 classes.
Found 23939 images belonging to 2 classes.


In [22]:
# Load the model
model_path = r'A:\AI DB\LSPD\models\inceptionv3_lspd_unfreezeBalanced.keras'
model = load_model(model_path)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
 flatten_3 (Flatten)         (None, 2048)              0         
                                                                 
 dense_8 (Dense)             (None, 512)               1049088   
                                                                 
 dense_9 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 22,854,437
Trainable params: 2,544,645
Non-trainable params: 20,309,792
_________________________________________________________________


In [ ]:
# Make predictions
ypred = model.predict(test_generator)

93/94 [============================>.] - ETA: 1s

In [ ]:
classes = ['drawing', 'hentai', 'normal', 'porn', 'sexy']

# Create a mapping: 0 for 'drawing' and 'normal', 1 for 'hentai', 'porn', 'sexy'
class_mapping = {'drawing': 0, 'hentai': 1, 'normal': 0, 'porn': 1, 'sexy': 1}

# Convert the ypred array to class indices
class_indices = np.argmax(ypred, axis=1)

# Convert class indices to 0 or 1 based on the mapping
converted_output = np.array([class_mapping[classes[index]] for index in class_indices])
converted_output

In [ ]:
# Convert predicted probabilities to binary predictions
#threshold = 0.5
#y_pred_classes = (ypred > threshold).astype(int)
y_pred_classes=converted_output
# Get actual classes
y_true_classes = test_generator.classes

# Create a confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Normalize the confusion matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Generate a classification report
cr = classification_report(y_true_classes, y_pred_classes, digits=4)
print("Classification Report:\n", cr)

# Plot the confusion matrix with annotations positioned at the bottom
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, cmap='YlGnBu', fmt='d', xticklabels=['Normal', 'Porno'], yticklabels=['Normal', 'Porno'], annot_kws={'size': 16})

# Add custom annotations at the bottom of each cell
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j + 0.5, i + 0.2, '{:.2f}%'.format(cm_normalized[i, j] * 100), ha='center', va='bottom', color='black', fontsize=14)

plt.ylabel('Actual', fontsize=12)
plt.xlabel('Predicted', fontsize=12)
plt.title('Confusion Matrix', fontsize=16)
plt.show()

# Calculate accuracy score
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print("Accuracy Score:", format(accuracy, '.4f'))

# Calculate F1 score
f1 = f1_score(y_true_classes, y_pred_classes)
print("F1 Score:", format(f1, '.4f'))